In [12]:
import nmmo

In [2]:
from nmmo.task.task_spec import TaskSpec

# Let's start with pre-built eval functions
from nmmo.task.base_predicates import CountEvent, InventorySpaceGE, TickGE, norm

In [3]:
# Here are very simple training tasks, based on a pre-built function, CountEvent
curriculum = []
# Agents have completed the task if they have done the event N times
essential_events = [
    "GO_FARTHEST",
    "EAT_FOOD",
    "DRINK_WATER",
    "SCORE_HIT",
    "HARVEST_ITEM",
    "LEVEL_UP",
]

for event_code in essential_events:
    curriculum.append(
        TaskSpec(
            eval_fn=CountEvent,  # is a pre-built eval function
            eval_fn_kwargs={"event": event_code, "N": 10},  # kwargs for CountEvent
        )
    )

print("Curriculum so far:\n", curriculum)
print("\nOne example training task:\n", curriculum[0])

Curriculum so far:
 [TaskSpec(eval_fn=<function CountEvent at 0x7f356c57cd30>, eval_fn_kwargs={'event': 'GO_FARTHEST', 'N': 10}, task_cls=<class 'nmmo.task.task_api.Task'>, task_kwargs={}, reward_to='agent', sampling_weight=1.0, embedding=None, predicate=None), TaskSpec(eval_fn=<function CountEvent at 0x7f356c57cd30>, eval_fn_kwargs={'event': 'EAT_FOOD', 'N': 10}, task_cls=<class 'nmmo.task.task_api.Task'>, task_kwargs={}, reward_to='agent', sampling_weight=1.0, embedding=None, predicate=None), TaskSpec(eval_fn=<function CountEvent at 0x7f356c57cd30>, eval_fn_kwargs={'event': 'DRINK_WATER', 'N': 10}, task_cls=<class 'nmmo.task.task_api.Task'>, task_kwargs={}, reward_to='agent', sampling_weight=1.0, embedding=None, predicate=None), TaskSpec(eval_fn=<function CountEvent at 0x7f356c57cd30>, eval_fn_kwargs={'event': 'SCORE_HIT', 'N': 10}, task_cls=<class 'nmmo.task.task_api.Task'>, task_kwargs={}, reward_to='agent', sampling_weight=1.0, embedding=None, predicate=None), TaskSpec(eval_fn=<fu

In [4]:
# You can also use pre-built eval functions to define your own function
def PracticeInventoryManagement(gs, subject, space, num_tick):
    return norm(InventorySpaceGE(gs, subject, space) * TickGE(gs, subject, num_tick))

# Training tasks are defined using TaskSpec
for space in [2, 4, 8]:
    curriculum.append(
        TaskSpec(
            eval_fn=PracticeInventoryManagement,
            eval_fn_kwargs={"space": space, "num_tick": 500},
        )
    )

In [5]:
# The eval functions can be built directly from accessing attributes in GameState and subject
# GameState: https://github.com/NeuralMMO/environment/blob/2.0/nmmo/task/game_state.py#L32

def PracticeEating(gs, subject):
    """The progress, the max of which is 1, should
    * increase small for each eating
    * increase big for the 1st and 3rd eating
    * reach 1 with 10 eatings
    """
    num_eat = len(subject.event.EAT_FOOD)
    progress = num_eat * 0.06
    if num_eat >= 1:
        progress += 0.1
    if num_eat >= 3:
        progress += 0.3
    return norm(progress)  # norm is a helper function to normalize the value to [0, 1]

curriculum.append(TaskSpec(eval_fn=PracticeEating, eval_fn_kwargs={}))

In [6]:
# Import the custom curriculum
import curriculum_generation.curriculum_tutorial as tutorial
CURRICULUM = tutorial.curriculum
print("The number of training tasks in the curriculum:", len(CURRICULUM))

The number of training tasks in the curriculum: 10


In [7]:
# Check if these task specs are valid in the nmmo environment
# Invalid tasks will crash your agent training
from nmmo.task.task_spec import check_task_spec

results = check_task_spec(CURRICULUM)
num_error = 0
for result in results:
  if result["runnable"] is False:
    print("ERROR: ", result["spec_name"])
    num_error += 1
assert num_error == 0, "Invalid task specs will crash training. Please fix them."
print("All training tasks are valid.")

All training tasks are valid.


In [8]:
# The task_spec must be picklable to be used for agent training
CURRICULUM_FILE_PATH = "custom_curriculum_with_embedding.pkl"
with open(CURRICULUM_FILE_PATH, "wb") as f:
  import dill
  dill.dump(CURRICULUM, f)
print("All training tasks are picklable.")

All training tasks are picklable.


In [11]:
# Check the available RAM before running below
import psutil
avail_ram = psutil.virtual_memory()[1]/1_000_000_000  # GB
print(avail_ram)
assert avail_ram > 16, "Need a high-ram instance, available with Colab Pro."



182.676545536


In [13]:
import os

In [18]:
os.path.dirname('.')

''